In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import BaseCrossValidator
from sklearn.linear_model import Ridge
import time
from sklearn.metrics import make_scorer
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
df = pd.read_excel('Data diolah darin.xlsx')
df

,Backhoe,HP(watt),Norm_MAP,RPM,Age,Load_Factor,Engine_Tier,TEMP[C],Fuel[g/s],NOx[g/s],...,NOx[g/hr],HC[g/hr],CO[g/hr],CO2[g/hr],PM[mg/hr],Nox (g/kl),HC (g/kl),CO (g/kl),CO2 (g/kl),PM (g/kl)
0,1,65621.6,0.000000,833,12,0.21,2,22,0.430000,0.024090,...,86.724000,25.848000,13.176000,4780.656000,14.4,3730.117500,1111.757728,566.717728,2.056225e+05,0.619362
1,1,65621.6,0.012346,800,12,0.21,2,22,0.620000,0.030320,...,109.152000,25.668000,16.164000,7008.264000,21.6,4694.776362,1104.015681,695.235681,3.014350e+05,0.929047
2,1,65621.6,0.012346,826,12,0.21,2,22,0.510000,0.030730,...,110.628000,10.980000,10.584000,5722.920000,21.6,4758.261138,472.264772,455.232272,2.461506e+05,0.929047
3,1,65621.6,0.012346,831,12,0.21,2,22,0.390522,0.023693,...,85.293664,20.747853,4.825082,4382.553038,21.6,3668.596780,892.393432,207.533355,1.884996e+05,0.929047
4,1,65621.6,0.012346,834,12,0.21,2,22,0.413024,0.026107,...,93.985262,6.979907,9.880556,4673.503069,21.6,4042.434273,300.215326,424.976193,2.010137e+05,0.929047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37523,8,72332.9,0.975904,1676,36,0.21,2,41,1.590000,0.058830,...,211.788000,19.908000,28.152000,17957.592000,1800.0,445343.100000,41862.100000,59197.400000,3.776083e+07,3785.000000
37524,8,72332.9,0.975904,1683,36,0.21,2,42,1.320000,0.059500,...,214.200000,19.800000,30.780000,14905.044000,1656.0,542545.340900,50151.250000,77962.397730,3.775286e+07,4194.468182
37525,8,72332.9,0.975904,1846,36,0.21,2,42,1.530000,0.059640,...,214.704000,21.744000,33.840000,17349.048000,1872.0,469179.694100,47515.850980,73948.509800,3.791183e+07,4090.768627
37526,8,72332.9,0.987952,1876,36,0.21,2,42,1.760000,0.056200,...,202.320000,22.356000,34.740000,19948.572000,1980.0,384340.943200,42468.990340,65994.485800,3.789568e+07,3761.343750


In [3]:
df.replace('#DIV/0!', np.nan, inplace=True)
df.dropna(inplace=True)

C:\Users\TI\AppData\Local\Temp\ipykernel_4428\2881342136.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('#DIV/0!', np.nan, inplace=True)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37527 entries, 0 to 37527
Data columns (total 30 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Backhoe      37527 non-null  int64  
 1   HP(watt)     37527 non-null  float64
 2   Norm_MAP     37527 non-null  float64
 3   RPM          37527 non-null  float64
 4   Age          37527 non-null  int64  
 5   Load_Factor  37527 non-null  float64
 6   Engine_Tier  37527 non-null  int64  
 7   TEMP[C]      37527 non-null  float64
 8   Fuel[g/s]    37527 non-null  float64
 9   NOx[g/s]     37527 non-null  float64
 10  HC[g/s]      37527 non-null  float64
 11  CO[g/s]      37527 non-null  float64
 12  CO2[g/s]     37527 non-null  float64
 13  PM[mg/s]     37527 non-null  float64
 14  Nox (g/l)    37527 non-null  float64
 15  HC (g/l)     37527 non-null  float64
 16  CO (g/l)     37527 non-null  float64
 17  CO2 (g/l)    37527 non-null  float64
 18  PM (g/l)     37527 non-null  float64
 19  Fuel[g/hr

In [5]:
X = df[['Backhoe', 'HP(watt)', 'Norm_MAP', 'RPM', 'Age', 'Load_Factor', 'Engine_Tier']]
y = df['HC[g/s]']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()

C:\Users\TI\AppData\Local\Temp\ipykernel_4428\1236518287.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_train = y_train.ravel()
C:\Users\TI\AppData\Local\Temp\ipykernel_4428\1236518287.py:4: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_test = y_test.ravel()


In [7]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(26268, 7)
(26268,)
(11259, 7)
(11259,)


GBR MODEL

In [8]:
gbr = GradientBoostingRegressor(loss='squared_error')

In [9]:
gbr.fit(X_train,y_train)
y_pred = gbr.predict(X_test)

In [28]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_test, y_pred))

def mape(y_true, y_pred):
    return mean_absolute_percentage_error(y_true, y_pred)

rmse_score = rmse(y_test, y_pred)
mape_score = mape(y_test, y_pred)

print(f'RMSE: {rmse_score:}')
print(f"MAPE: {mape_score:.2f}%")

RMSE: 0.0016052537520657546
MAPE: 12871839039.62%


GRID

In [11]:
params = {
    'learning_rate': np.arange(0.1, 1.0, 0.1),
    'n_estimators': np.arange(1, 51, 5),
    'subsample': np.arange(0.5, 1.0, 0.1),
    'min_samples_split': range(2, 11),
    'min_samples_leaf': range(1, 11),
    'max_depth': range(1, 26)
}

rmse_scorer = make_scorer(mean_squared_error, squared=False)
mape_scorer = make_scorer(mean_absolute_percentage_error)

In [12]:
grid = GridSearchCV(
    estimator=gbr,
    param_grid=params,
    scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
    refit='RMSE', 
    cv=2,  
    verbose=3,
    n_jobs=-1
)

start_time = time.time()
gbr_result = grid.fit(X_train,y_train)

print("Best: %f using %s" % (grid.best_score_, grid.best_params_))
print("Best RMSE:", grid.best_score_)
print("Best MAPE:", grid.cv_results_['mean_test_MAPE'][grid.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Fitting 2 folds for each of 1012500 candidates, totalling 2025000 fits
Best: 0.003489 using {'learning_rate': 0.9, 'max_depth': 19, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 46, 'subsample': 0.5}
Best RMSE: 0.0034891452339358407
Best MAPE: 8388678434.40439
Execution time: 56164.896951437 s


In [13]:
best_grid = grid.best_estimator_
y_pred_grid = best_grid.predict(X_test)
rmse_grid = np.sqrt(mean_squared_error(y_test, y_pred_grid))
mape_grid = mean_absolute_percentage_error(y_test, y_pred_grid)
print(f"RMSE: {rmse_grid}")
print(f"MAPE: {mape_grid:.2f}%")

RMSE: 0.0034695010407030246
MAPE: 18531935392.85%


In [14]:
# export to excel
pd.DataFrame(grid.cv_results_).to_excel('GBR_GS_unscal_HC.xlsx', index=False)

random search

In [15]:
rs = RandomizedSearchCV(
    gbr,
    params,
    scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
    refit='RMSE',
    n_iter=100,
    cv=5,  
    verbose=0,
    n_jobs=-1
)

start_time = time.time()
rs_result = rs.fit(X_train,y_train)

print("Best: %f using %s" % (rs.best_score_, rs.best_params_))
print("Best RMSE:", rs.best_score_)
print("Best MAPE:", rs.cv_results_['mean_test_MAPE'][rs.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Best: 0.002047 using {'subsample': 0.7, 'n_estimators': 1, 'min_samples_split': 7, 'min_samples_leaf': 10, 'max_depth': 1, 'learning_rate': 0.7000000000000001}
Best RMSE: 0.0020466646614763903
Best MAPE: 11684113789.806808
Execution time: 18.20041799545288 s


In [16]:
best_rs = rs.best_estimator_
y_pred_rs = best_rs.predict(X_test)
rmse_rs = np.sqrt(mean_squared_error(y_test, y_pred_rs))
mape_rs = mean_absolute_percentage_error(y_test, y_pred_rs)
print(f"RMSE: {rmse_rs}")
print(f"MAPE: {mape_rs:.2f}%")

RMSE: 0.0021338758369145094
MAPE: 11743585545.86%


In [17]:
rs.cv_results_

{'mean_fit_time': array([0.96843824, 0.30919476, 1.04673805, 0.39973383, 0.2980299 ,
        0.24957552, 0.05707645, 0.47307305, 0.70357828, 0.83211613,
        0.29670601, 0.79421511, 0.42162447, 0.32653751, 0.00787034,
        0.48591342, 1.70294204, 0.97935734, 0.02687044, 1.03947997,
        0.25925841, 0.84402142, 1.4085391 , 0.6058176 , 0.27440801,
        1.27538567, 0.0639699 , 0.62361593, 0.02291031, 0.01906762,
        0.01100731, 0.43959231, 0.38423862, 0.23605757, 0.97011094,
        0.01057487, 1.63890653, 0.24346924, 0.56550975, 0.4627151 ,
        0.01160803, 0.2764328 , 0.31719661, 1.01490521, 0.3262466 ,
        0.18795705, 0.02113194, 0.2235487 , 0.17178702, 1.18021526,
        1.5751791 , 0.84539089, 0.16355562, 2.05019054, 0.32973504,
        0.85822434, 0.0317008 , 1.11291947, 0.02666793, 1.08380809,
        1.379491  , 0.62178779, 0.18632588, 1.91873446, 0.12078443,
        0.69016881, 0.15394249, 0.19799013, 1.72719488, 0.24957662,
        0.49913993, 0.6625607 ,

In [18]:
pd.DataFrame(rs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_depth,param_learning_rate,...,std_test_RMSE,rank_test_RMSE,split0_test_MAPE,split1_test_MAPE,split2_test_MAPE,split3_test_MAPE,split4_test_MAPE,mean_test_MAPE,std_test_MAPE,rank_test_MAPE
0,0.968438,0.057576,0.016235,0.003491,0.6,41,7,7,12,0.3,...,0.000027,83,9.867953e+09,1.209196e+10,1.048716e+10,1.052945e+10,6.540906e+09,9.903486e+09,1.834673e+09,51
1,0.309195,0.015668,0.006166,0.005478,0.7,36,7,9,3,0.8,...,0.000054,56,7.687356e+09,1.114846e+10,1.078107e+10,1.169985e+10,7.052289e+09,9.673805e+09,1.914358e+09,70
2,1.046738,0.017749,0.016893,0.005846,0.9,26,10,1,19,0.2,...,0.000027,42,8.425576e+09,1.280121e+10,1.060293e+10,1.028519e+10,6.048557e+09,9.632693e+09,2.267589e+09,75
3,0.399734,0.023600,0.007244,0.004188,0.6,21,8,10,9,0.2,...,0.000025,98,8.518464e+09,1.157889e+10,1.032016e+10,1.047512e+10,6.154542e+09,9.409435e+09,1.900515e+09,86
4,0.298030,0.011229,0.008583,0.001976,0.5,11,3,1,17,0.4,...,0.000031,26,5.693762e+09,1.094771e+10,1.106671e+10,1.033523e+10,6.378272e+09,8.884338e+09,2.348851e+09,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.082404,0.042792,0.013706,0.003642,0.9,21,7,5,18,0.2,...,0.000035,85,9.798992e+09,1.287217e+10,1.063490e+10,1.007888e+10,7.531223e+09,1.018323e+10,1.710326e+09,36
96,0.688162,0.038620,0.013085,0.002546,0.5,26,5,9,12,0.4,...,0.000033,74,9.701691e+09,1.140758e+10,1.029649e+10,1.007885e+10,7.580139e+09,9.812951e+09,1.252611e+09,60
97,0.051420,0.002441,0.002015,0.004030,0.7,1,6,3,25,0.7,...,0.000040,47,9.619103e+09,1.517982e+10,1.119904e+10,1.110166e+10,8.743900e+09,1.116871e+10,2.207957e+09,12
98,0.097321,0.009481,0.006758,0.004189,0.6,16,4,3,1,0.7,...,0.000055,10,8.885312e+09,1.267572e+10,9.490665e+09,1.423210e+10,7.440077e+09,1.054478e+10,2.515608e+09,20


In [19]:
# export to excel
pd.DataFrame(rs.cv_results_).to_excel('GBR_RS_unscal_HC.xlsx', index=False)

BAYESSIAN

In [20]:
def gb_regression_cv(learning_rate,n_estimators, subsample, min_samples_split, min_samples_leaf, max_depth):
    gbr = GradientBoostingRegressor(
        learning_rate=learning_rate,
        n_estimators=int(n_estimators),
        subsample=(subsample), 
        min_samples_split=int(min_samples_split), 
        min_samples_leaf=int(min_samples_leaf),
        max_depth=int(max_depth)
    )
    scores = cross_val_score(gbr, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse = np.sqrt(-scores.mean())
    return -rmse  # Return only the RMSE score

In [21]:
bayes_opt = BayesianOptimization(
    gb_regression_cv,
    {
        'learning_rate': (0.01, 1.0),
        'n_estimators': (1, 51),
        'subsample': (0.5, 1.0),
        'min_samples_split': (2, 11),
        'min_samples_leaf': (1, 11),
        'max_depth': (1, 26),
    }
)

In [22]:
start_time = time.time()
bayes_opt.maximize(init_points=10, n_iter=100)
print("Execution time: " + str((time.time() - start_time)) + ' s')

|   iter    |  target   | learni... | max_depth | min_sa... | min_sa... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
| 1         | -0.001474 | 0.1012    | 18.59     | 8.827     | 6.903     | 15.73     | 0.9393    |
| 2         | -0.001729 | 0.4376    | 4.642     | 1.324     | 3.636     | 3.253     | 0.6024    |
| 3         | -0.001565 | 0.4862    | 19.88     | 3.045     | 10.27     | 8.3       | 0.9438    |
| 4         | -0.001514 | 0.482     | 9.884     | 5.611     | 4.028     | 24.46     | 0.5562    |
| 5         | -0.001573 | 0.8982    | 3.074     | 8.007     | 2.487     | 28.96     | 0.758     |
| 6         | -0.001485 | 0.4232    | 13.3      | 9.436     | 10.56     | 3.244     | 0.7134    |
| 7         | -0.001504 | 0.1367    | 20.42     | 6.495     | 6.584     | 9.051     | 0.9764    |
| 8         | -0.001736 | 0.5175    | 21.52     | 2.578     | 6.456     | 25.73     | 0.6967    |
| 9         | -0.001

In [23]:
best_iter = np.argmin([res['target'] for res in bayes_opt.res])

In [24]:
best_params = bayes_opt.res[best_iter]['params']

print("Best Parameters:", best_params)

Best Parameters: {'learning_rate': 0.03634739471998861, 'max_depth': 2.683223690502084, 'min_samples_leaf': 5.751791416687816, 'min_samples_split': 10.22916897535856, 'n_estimators': 6.821772414508642, 'subsample': 0.5324196514538198}


In [25]:
best_bs = GradientBoostingRegressor(
    learning_rate=(best_params['learning_rate']),
    n_estimators=int(best_params['n_estimators']),
    subsample=(best_params['subsample']), 
    min_samples_split=int(best_params['min_samples_split']), 
    min_samples_leaf=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']),
    loss='squared_error'
)
best_bs.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.03634739471998861, max_depth=2,
                          min_samples_leaf=6, min_samples_split=10,
                          n_estimators=6, subsample=0.5324196514538198)

In [26]:
y_pred_bs = best_bs.predict(X_test)
rmse_bs = np.sqrt(mean_squared_error(y_test, y_pred_bs))
mape_bs = mean_absolute_percentage_error(y_test, y_pred_bs)

print(f"RMSE: {rmse_bs}")
print(f"MAPE: {mape_bs:.2f}%")

RMSE: 0.002167431599986449
MAPE: 11482583492.06%
